# Fuel Prices by Brands

In this notebook, we can see the price behaviour for different brands in Germany using data from July 2018 to June 2019. 

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date

import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import iplot, init_notebook_mode
from IPython.display import Image, IFrame, display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual


# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 10

In [2]:
data_brands_months = pd.read_parquet('data_brands_months.parquet')

In [3]:
data_brands_months['date_str'] = data_brands_months.apply(lambda x: str(x.year) + " - " + str(x.month), axis=1)
data_brands_months['date'] = data_brands_months.apply(lambda x: datetime(x.year, x.month, 1), axis=1)

In [4]:
top_brands = ['SHELL', 
            'Esso',
            'AVIA',
            'OMV',
            'Westfalen',
            'ARAL',
            'TOTAL',
            'JET',
            'STAR',
            'Agip']
data_top_brands_months = data_brands_months.loc[data_brands_months['m.brand_name'].isin(top_brands), :]

## Prices for top brands

Average of the prices per brand in a period of time.

Please keep in mind that brands have different prices strategies, which means that some of their stations are much cheaper and other much more expensive according to their location and the services they offer. As the values below are averages i.e there are stations from one brand that could be more expensive than a station from one brand with a higher average. 

Select the time range to view the mean prices for each fuel type. 

In [5]:
start_date = date(2018, 7, 1)
end_date = date(2019, 6, 30)

dates = pd.date_range(start_date, end_date, freq='MS')

options = [(date.strftime(' %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

In [6]:
@interact
def mean_per_timeperiod_top_brands(date_range=selection_range_slider):
    display(HTML(f'<h2>Showing average prices for top brands and fuel types<h3>'))
    filter_dates = (data_top_brands_months['date'] <= date_range[1]) & (data_top_brands_months['date'] >= date_range[0])
    mean_prices = data_top_brands_months.loc[filter_dates].groupby(['m.brand_name', 'fuel_type'])['mean'].mean().reset_index()
    mean_prices = mean_prices.pivot(index='m.brand_name', values='mean', columns='fuel_type')
    mean_prices['total_mean'] = mean_prices.mean(axis=1, numeric_only=True)
    mean_prices = mean_prices.sort_values('total_mean')
    display(mean_prices[['diesel', 'e5', 'e10']])

interactive(children=(SelectionRangeSlider(description='Dates', index=(0, 11), layout=Layout(width='500px'), o…

### Plot: Timeline to compare 2 brands

Here, you can choose two different brands and compare the history for a fuel type using a descriptive (minimum, mean, median and maximum).

In [7]:
@interact
def scatter_plot(brand_base=data_top_brands_months['m.brand_name'].unique(), brand=data_top_brands_months['m.brand_name'].unique(), fuel_type=data_top_brands_months['fuel_type'].unique(), descriptive=['median', 'mean', 'min', 'max']):
    data_base = data_top_brands_months.loc[(data_top_brands_months['m.brand_name'] == brand_base) & (data_top_brands_months['fuel_type'] == fuel_type), ['date_str', 'min', 'median', 'mean', 'max']]
    data = data_top_brands_months.loc[(data_top_brands_months['m.brand_name'] == brand) & (data_top_brands_months['fuel_type'] == fuel_type), ['date_str', 'min', 'median', 'mean', 'max']]
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=data_base['date_str'], y=data_base[descriptive], name=brand_base))
    fig.add_trace(go.Scatter(x=data['date_str'], y=data[descriptive], name=brand))
    fig.update_layout(title_text='Time Series', xaxis_rangeslider_visible=True)
    fig_widget = go.FigureWidget(fig)
    iplot(fig)

interactive(children=(Dropdown(description='brand_base', options=('ARAL', 'AVIA', 'Agip', 'Esso', 'JET', 'OMV'…

## Table: Prices for top brands and fuel type

Filtering a brand and fuel type, you can see the minimum value, median, mean (average) and maximum price for each month.

In [8]:
@interact
def show_prices_per_top_brand(brand=data_top_brands_months['m.brand_name'].unique(), fuel_type=data_top_brands_months['fuel_type'].unique()):
    display(HTML(f'<h2>Showing prices for brand {brand} and fuel type {fuel_type}<h3>'))
    display(data_top_brands_months.loc[(data_top_brands_months['m.brand_name'] == brand) & (data_top_brands_months['fuel_type'] == fuel_type), ['year', 'month', 'median', 'mean']])

interactive(children=(Dropdown(description='brand', options=('ARAL', 'AVIA', 'Agip', 'Esso', 'JET', 'OMV', 'SH…

## Appendix : Prices for all brands

In [9]:
@interact
def show_prices_per_brand(brand=data_brands_months['m.brand_name'].unique(), fuel_type=data_brands_months['fuel_type'].unique()):
    display(HTML(f'<h2>Showing prices for brand {brand} and fuel type {fuel_type}<h3>'))
    display(data_brands_months.loc[(data_brands_months['m.brand_name'] == brand) & (data_brands_months['fuel_type'] == fuel_type), ['year', 'month', 'median', 'mean']])

interactive(children=(Dropdown(description='brand', options=('', ' AVIA Xpress ', ' Autohof Frechen (Mundorf T…